In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-1f3e5gph
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-1f3e5gph
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 781ff5b76ba6c4c2d80dcbbec9983e147613cc71
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.1.0-py3-none-any.whl size=8011 sha256=e2b2acbb79d726227ace2ffdeb18b901e4debb9cf9419b13a35a389a655ee4ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-i8hclcq1/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [ ]:
%load_ext nvcc4jupyter

Source files will be saved in "/tmp/tmpxoiaf_6z".


write a program to sum of two numbers using cuda

In [ ]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
__global__ void add(int *a, int *b, int *c) {
*c = *a + *b;
}
int main() {
int a, b, c;
int *d_a, *d_b, *d_c;
int size = sizeof(int);
cudaMalloc((void **)&d_a, size);
cudaMalloc((void **)&d_b, size);
cudaMalloc((void **)&d_c, size);
c = 0;
a = 4;
b = 5;

cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);

add<<<1,1>>>(d_a, d_b, d_c);

cudaError err = cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);
  if(err!=cudaSuccess) {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  }
printf("result is %d\n",c);

cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);
return 0;
}

result is 9



Write a program to multiply square matrices using CUDA.(N*N)

In [ ]:
%%cuda
#include <stdio.h>

#define N 2 // size of the matrix (N x N)

__global__ void matrixMul(int *a, int *b, int *c) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    int temp_sum = 0;
    if (row < N && col < N) {
        for (int i = 0; i < N; i++) {
            temp_sum += a[row * N + i] * b[i * N + col];
        }
        c[row * N + col] = temp_sum;
    }
}

int main() {
    int size = N * N * sizeof(int);
    int a[N*N] = {1, 2, 3, 4};
    int b[N*N] = {16, 15, 14, 13};
    int c[N*N];
    int *d_a, *d_b, *d_c;

    // allocate memory on the device
    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_b, size);
    cudaMalloc((void **)&d_c, size);

    // copy inputs to device
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    // launch the kernel
    dim3 threadsPerBlock(N, N);
    dim3 blocksPerGrid(1, 1);
    if (N*N > 512){
        threadsPerBlock.x = 512;
        threadsPerBlock.y = 512;
        blocksPerGrid.x = ceil(double(N)/double(threadsPerBlock.x));
        blocksPerGrid.y = ceil(double(N)/double(threadsPerBlock.y));
    }

    matrixMul<<<blocksPerGrid,threadsPerBlock>>>(d_a, d_b, d_c);

    // copy result back to host
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

    // print the result
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            printf("%d ", c[i * N + j]);
        }
        printf("\n");
    }

    // free memory on the device
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}


44 41 
104 97 



In [ ]:
%%cuda
#include <stdio.h>

#define N 4 // size of the matrix (N x N)

__global__ void matrixMul(int *a, int *b, int *c) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    int temp_sum = 0;
    if (row < N && col < N) {
        for (int i = 0; i < N; i++) {
            temp_sum += a[row * N + i] * b[i * N + col];
        }
        c[row * N + col] = temp_sum;
    }
}

int main() {
    int size = N * N * sizeof(int);
    int a[N*N] = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16};
    int b[N*N] = {16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1};
    int c[N*N];
    int *d_a, *d_b, *d_c;

    // allocate memory on the device
    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_b, size);
    cudaMalloc((void **)&d_c, size);

    // copy inputs to device
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    // launch the kernel
    dim3 threadsPerBlock(N, N);
    dim3 blocksPerGrid(1, 1);
    if (N*N > 512){
        threadsPerBlock.x = 512;
        threadsPerBlock.y = 512;
        blocksPerGrid.x = ceil(double(N)/double(threadsPerBlock.x));
        blocksPerGrid.y = ceil(double(N)/double(threadsPerBlock.y));
    }

    matrixMul<<<blocksPerGrid,threadsPerBlock>>>(d_a, d_b, d_c);

    // copy result back to host
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

    // print the result
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            printf("%d ", c[i * N + j]);
        }
        printf("\n");
    }

    // free memory on the device
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}


80 70 60 50 
240 214 188 162 
400 358 316 274 
560 502 444 386 



Write a program to multiply two matrices using CUDA.

In [ ]:
%%cuda
#include <stdio.h>

#define M 2 // number of rows in the first matrix and the result matrix
#define N 3 // number of columns in the first matrix and rows in the second matrix
#define P 3 // number of columns in the second matrix and the result matrix

__global__ void matrixMul(int *a, int *b, int *c) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    int temp_sum = 0;
    if (row < M && col < P) {
        for (int i = 0; i < N; i++) {
            temp_sum += a[row * N + i] * b[i * P + col];
        }
        c[row * P + col] = temp_sum;
    }
}

int main() {
    int sizeA = M * N * sizeof(int);
    int sizeB = N * P * sizeof(int);
    int sizeC = M * P * sizeof(int);
    int a[M*N] = {1, 1, 3, 0, 3, 8};
    int b[N*P] = {1, 9, 6, 0, 1, 2};
    int c[M*P];
    int *d_a, *d_b, *d_c;

    // allocate memory on the device
    cudaMalloc((void **)&d_a, sizeA);
    cudaMalloc((void **)&d_b, sizeB);
    cudaMalloc((void **)&d_c, sizeC);

    // copy inputs to device
    cudaMemcpy(d_a, a, sizeA, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, sizeB, cudaMemcpyHostToDevice);

    // launch the kernel
    dim3 threadsPerBlock(N, N);
    dim3 blocksPerGrid(1, 1);
    if (N*N > 512){
        threadsPerBlock.x = 512;
        threadsPerBlock.y = 512;
        blocksPerGrid.x = ceil(double(N)/double(threadsPerBlock.x));
        blocksPerGrid.y = ceil(double(N)/double(threadsPerBlock.y));
    }

    matrixMul<<<blocksPerGrid,threadsPerBlock>>>(d_a, d_b, d_c);

    // copy result back to host
    cudaMemcpy(c, d_c, sizeC, cudaMemcpyDeviceToHost);

    // print the result
    for (int i = 0; i < M; i++) {
        for (int j = 0; j < P; j++) {
            printf("%d ", c[i * P + j]);
        }
        printf("\n");
    }

    // free memory on the device
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}


1 10 8 
0 3 6 



In [ ]:
%%cuda
//main program
#include <stdio.h>

#define M 3 // number of rows in the first matrix and the result matrix
#define N 4 // number of columns in the first matrix and rows in the second matrix
#define P 5 // number of columns in the second matrix and the result matrix

__global__ void matrixMul(int *a, int *b, int *c) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    int temp_sum = 0;
    if (row < M && col < P) {
        for (int i = 0; i < N; i++) {
            temp_sum += a[row * N + i] * b[i * P + col];
        }
        c[row * P + col] = temp_sum;
    }
}

int main() {
    int sizeA = M * N * sizeof(int);
    int sizeB = N * P * sizeof(int);
    int sizeC = M * P * sizeof(int);
    int a[M*N] = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12};
    int b[N*P] = {1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4};
    int c[M*P];
    int *d_a, *d_b, *d_c;

    // allocate memory on the device
    cudaMalloc((void **)&d_a, sizeA);
    cudaMalloc((void **)&d_b, sizeB);
    cudaMalloc((void **)&d_c, sizeC);

    // copy inputs to device
    cudaMemcpy(d_a, a, sizeA, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, sizeB, cudaMemcpyHostToDevice);

    // launch the kernel
    dim3 threadsPerBlock(16, 16); // Assuming a maximum of 256 threads per block
    dim3 blocksPerGrid((P + threadsPerBlock.x - 1) / threadsPerBlock.x, (M + threadsPerBlock.y - 1) / threadsPerBlock.y);

    matrixMul<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c);

    // copy result back to host
    cudaMemcpy(c, d_c, sizeC, cudaMemcpyDeviceToHost);

    // print the result
    for (int i = 0; i < M; i++) {
        for (int j = 0; j < P; j++) {
            printf("%d ", c[i * P + j]);
        }
        printf("\n");
    }

    // free memory on the device
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}


30 30 30 30 30 
70 70 70 70 70 
110 110 110 110 110 



 Write a CUDA program to find the sum of elements of an array using parallel reduction.

In [ ]:
%%cuda
#include <iostream>
#include <cuda_runtime.h>

const int N = 16; // Number of elements in the array

__global__ void reduceSum(int* input, int* output, int n) {
    extern __shared__ int sharedMem[];

    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Load data from global memory to shared memory
    sharedMem[tid] = (idx < n) ? input[idx] : 0;
    __syncthreads();

    // Perform parallel reduction
    for (int stride = 1; stride < blockDim.x; stride *= 2) {
        int index = 2 * stride * tid;
        if (index < blockDim.x) {
            sharedMem[index] += sharedMem[index + stride];
        }
        __syncthreads();
    }

    // Write the result to output array
    if (tid == 0) {
        output[blockIdx.x] = sharedMem[0];
    }
}

int main() {
    int array[N] = {10, 1, 8, -1, 0, -2, 3, 5, -2, -3, 2, 7, 0, 11, 0, 2};
    int* d_input, * d_output;

    // Allocate memory on the device
    cudaMalloc((void**)&d_input, N * sizeof(int));
    cudaMalloc((void**)&d_output, sizeof(int));

    // Copy the array from host to device
    cudaMemcpy(d_input, array, N * sizeof(int), cudaMemcpyHostToDevice);

    // Define grid and block sizes
    dim3 gridSize(1, 1, 1);
    dim3 blockSize(N, 1, 1);

    // Timing variables
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Record the start event
    cudaEventRecord(start);

    // Launch the kernel
    reduceSum<<<gridSize, blockSize, N * sizeof(int)>>>(d_input, d_output, N);

    // Record the stop event
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    // Copy the result back to the host
    int result;
    cudaMemcpy(&result, d_output, sizeof(int), cudaMemcpyDeviceToHost);

    // Print the result
    std::cout << "Sum: " << result << std::endl;

     // Calculate and print the execution time
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    std::cout << "Execution Time: " << milliseconds << " ms" << std::endl;

    // Free allocated memory on the device
    cudaFree(d_input);
    cudaFree(d_output);

    // Destroy events
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}


Sum: 41
Execution Time: 41.5871 ms



In [ ]:
%%cuda
#include <stdio.h>

// CUDA kernel to perform reduction operation
__global__ void sum(int *input, int *output, int n) {
    extern __shared__ int shared[];

    int tid = threadIdx.x;
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    if (i < n) {
        shared[tid] = input[i];
    } else {
        shared[tid] = 0;
    }

    __syncthreads();

    for (int stride = 1; stride < blockDim.x; stride *= 2) {
        if (tid % (2 * stride) == 0) {
            shared[tid] += shared[tid + stride];
        }

        __syncthreads();
    }

    if (tid == 0) {
        output[blockIdx.x] = shared[0];
    }
}

// Host function to initialize array elements and launch kernel
int main() {
    int n = 100;
    int *input, *output;
    int size = n * sizeof(int);

    // Allocate memory on the host
    input = (int*)malloc(size);
    output = (int*)malloc(size);

    // Initialize array elements
    for (int i = 0; i < n; i++) {
        input[i] = i;
    }

    // Allocate memory on the device
    int *d_input, *d_output;
    cudaMalloc((void**)&d_input, size);
    cudaMalloc((void**)&d_output, size);

    // Copy input array to device
    cudaMemcpy(d_input, input, size, cudaMemcpyHostToDevice);

    // Timing variables
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Record the start event
    cudaEventRecord(start);

    // Launch kernel
    int blockSize = 256;
    int gridSize = (n + blockSize - 1) / blockSize;
    sum<<<gridSize, blockSize, blockSize * sizeof(int)>>>(d_input, d_output, n);

    // Record the stop event
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    // Copy result back to host
    cudaMemcpy(output, d_output, size, cudaMemcpyDeviceToHost);

    // Calculate final sum
    int final_sum = 0;
    for (int i = 0; i < gridSize; i++) {
        final_sum += output[i];
    }

    // Print final sum
    printf("Sum: %d\n", final_sum);

    // Calculate and print the execution time
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    printf("Execution Time: %.6f ms\n", milliseconds);

    // Free memory
    free(input);
    free(output);
    cudaFree(d_input);
    cudaFree(d_output);

    // Destroy events
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}


Sum: 4950
Execution Time: 49.958080 ms



 Write a CUDA program to find the minimum element in an array using parallel reduction.

In [ ]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <float.h>
#include <sys/time.h>

#define BLOCK_SIZE 256

// Kernel for parallel reduction
__global__ void minParallelReduction(float *input, float *output, int n) {
    __shared__ float sdata[BLOCK_SIZE];

    unsigned int tid = threadIdx.x;
    unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;

    sdata[tid] = (i < n) ? input[i] : FLT_MAX;
    __syncthreads();

    for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            sdata[tid] = fminf(sdata[tid], sdata[tid + s]);
        }
        __syncthreads();
    }

    if (tid == 0) {
        output[blockIdx.x] = sdata[0];
    }
}

// Serial reduction on CPU
float minSerialReduction(float *input, int n) {
    float min_val = FLT_MAX;
    for (int i = 0; i < n; ++i) {
        min_val = fminf(min_val, input[i]);
    }
    return min_val;
}

int main() {
    int n = 7; // Size of the array
    float h_input[] = {10, 21, 13, 41, 55, 76, 7};
    float *d_input, *d_output;
    float *h_output_parallel, *h_output_serial;
    cudaEvent_t start, stop;
    float elapsedTimeParallel, elapsedTimeSerial;

    // Allocate host memory
    h_output_parallel = (float *)malloc(sizeof(float));
    h_output_serial = (float *)malloc(sizeof(float));

    // Allocate device memory
    cudaMalloc((void **)&d_input, n * sizeof(float));
    cudaMalloc((void **)&d_output, sizeof(float));

    // Copy host input array to device
    cudaMemcpy(d_input, h_input, n * sizeof(float), cudaMemcpyHostToDevice);

    // Run parallel reduction
    dim3 blockDim(BLOCK_SIZE, 1, 1);
    dim3 gridDim((n + blockDim.x - 1) / blockDim.x, 1, 1);

    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start, 0);

    minParallelReduction<<<gridDim, blockDim>>>(d_input, d_output, n);
    cudaDeviceSynchronize();

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&elapsedTimeParallel, start, stop);

    // Copy result back to host
    cudaMemcpy(h_output_parallel, d_output, sizeof(float), cudaMemcpyDeviceToHost);

    // Run serial reduction
    cudaEventRecord(start, 0);

    *h_output_serial = minSerialReduction(h_input, n);

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&elapsedTimeSerial, start, stop);

    // Compare results
    printf("Minimum value (serial reduction): %f\n", *h_output_parallel);
    printf("Minimum value (parallel reduction): %f\n", *h_output_serial);

    // Print execution times
    printf("Execution time (serial reduction): %f ms\n", elapsedTimeParallel);
    printf("Execution time (parallel reduction): %f ms\n", elapsedTimeSerial);

    // Free device memory
    cudaFree(d_input);
    cudaFree(d_output);

    // Free host memory
    free(h_output_parallel);
    free(h_output_serial);

    return 0;
}


Write a CUDA program to implement 1D convolution.

In [ ]:
%%cuda
#include <stdio.h>

#define MASK_WIDTH 3
#define DATA_SIZE 8
#define TILE_WIDTH 4

__constant__ int mask[MASK_WIDTH];

__global__ void convolution_1d(int *input, int *output, int width) {
    __shared__ int N_ds[TILE_WIDTH + MASK_WIDTH - 1];
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int temp = 0;

    // Load input data into shared memory
    if (i >= 0 && i < width)
        N_ds[threadIdx.x + MASK_WIDTH/2] = input[i];
    else
        N_ds[threadIdx.x + MASK_WIDTH/2] = 0;

    // Load halo elements
    if (threadIdx.x < MASK_WIDTH/2) {
        if (i - MASK_WIDTH/2 >= 0)
            N_ds[threadIdx.x] = input[i - MASK_WIDTH/2];
        else
            N_ds[threadIdx.x] = 0;
        if (i + blockDim.x < width)
            N_ds[threadIdx.x + TILE_WIDTH + MASK_WIDTH/2] = input[i + blockDim.x];
        else
            N_ds[threadIdx.x + TILE_WIDTH + MASK_WIDTH/2] = 0;
    }

    __syncthreads();

    // Convolution
    for (int j = 0; j < MASK_WIDTH; j++)
        temp += N_ds[threadIdx.x + j] * mask[j];

    output[i] = temp;
}

int main() {
    int input[DATA_SIZE] = {1, 2, 3, 4, 5, 6, 7, 8};
    int mask_value[MASK_WIDTH] = {1, 0, -1};
    int output[DATA_SIZE] = {0};

    int *d_input, *d_output;

    cudaMalloc(&d_input, DATA_SIZE * sizeof(int));
    cudaMalloc(&d_output, DATA_SIZE * sizeof(int));
    cudaMemcpy(d_input, input, DATA_SIZE * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpyToSymbol(mask, mask_value, MASK_WIDTH * sizeof(int));

    convolution_1d<<<1, DATA_SIZE>>>(d_input, d_output, DATA_SIZE);

    cudaMemcpy(output, d_output, DATA_SIZE * sizeof(int), cudaMemcpyDeviceToHost);

    for (int i = 0; i < DATA_SIZE; i++)
        printf("%d ", output[i]);
    printf("\n");

    cudaFree(d_input);
    cudaFree(d_output);

    return 0;
}


Write a CUDA program to implement 2D convolution.

In [ ]:
%%cuda
#include <stdio.h>

#define MASK_WIDTH 3
#define DATA_SIZE_X 5
#define DATA_SIZE_Y 5
#define TILE_WIDTH_X 3
#define TILE_WIDTH_Y 3

__constant__ int mask[MASK_WIDTH][MASK_WIDTH];

__global__ void convolution_2d(int *input, int *output, int width, int height) {
    __shared__ int N_ds[TILE_WIDTH_Y + MASK_WIDTH - 1][TILE_WIDTH_X + MASK_WIDTH - 1];
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;
    int temp = 0;

    // Load input data into shared memory
    if (i >= 0 && i < width && j >= 0 && j < height)
        N_ds[threadIdx.y + MASK_WIDTH/2][threadIdx.x + MASK_WIDTH/2] = input[j * width + i];
    else
        N_ds[threadIdx.y + MASK_WIDTH/2][threadIdx.x + MASK_WIDTH/2] = 0;

    // Load halo elements
    if (threadIdx.x < MASK_WIDTH/2) {
        if (i - MASK_WIDTH/2 >= 0)
            N_ds[threadIdx.y + MASK_WIDTH/2][threadIdx.x] = input[j * width + (i - MASK_WIDTH/2)];
        else
            N_ds[threadIdx.y + MASK_WIDTH/2][threadIdx.x] = 0;
        if (i + blockDim.x < width)
            N_ds[threadIdx.y + MASK_WIDTH/2][threadIdx.x + TILE_WIDTH_X + MASK_WIDTH/2] = input[j * width + (i + blockDim.x)];
        else
            N_ds[threadIdx.y + MASK_WIDTH/2][threadIdx.x + TILE_WIDTH_X + MASK_WIDTH/2] = 0;
    }
    if (threadIdx.y < MASK_WIDTH/2) {
        if (j - MASK_WIDTH/2 >= 0)
            N_ds[threadIdx.y][threadIdx.x + MASK_WIDTH/2] = input[(j - MASK_WIDTH/2) * width + i];
        else
            N_ds[threadIdx.y][threadIdx.x + MASK_WIDTH/2] = 0;
        if (j + blockDim.y < height)
            N_ds[threadIdx.y + TILE_WIDTH_Y + MASK_WIDTH/2][threadIdx.x + MASK_WIDTH/2] = input[(j + blockDim.y) * width + i];
        else
            N_ds[threadIdx.y + TILE_WIDTH_Y + MASK_WIDTH/2][threadIdx.x + MASK_WIDTH/2] = 0;
    }
    __syncthreads();

    // Convolution
    for (int y = 0; y < MASK_WIDTH; y++) {
        for (int x = 0; x < MASK_WIDTH; x++) {
            temp += N_ds[threadIdx.y + y][threadIdx.x + x] * mask[y][x];
        }
    }

    if (i < width && j < height)
        output[j * width + i] = temp;
}

int main() {
    int input[DATA_SIZE_Y][DATA_SIZE_X] = {
        {1, 2, 3, 4, 5},
        {6, 7, 8, 9, 10},
        {11, 12, 13, 14, 15},
        {16, 17, 18, 19, 20},
        {21, 22, 23, 24, 25}
    };
    int mask_value[MASK_WIDTH][MASK_WIDTH] = {
        {1, 0, -1},
        {2, 0, -2},
        {1, 0, -1}
    };
    int output[DATA_SIZE_Y][DATA_SIZE_X] = {0};

    int *d_input, *d_output;

    cudaMalloc(&d_input, DATA_SIZE_X * DATA_SIZE_Y * sizeof(int));
    cudaMalloc(&d_output, DATA_SIZE_X * DATA_SIZE_Y * sizeof(int));
    cudaMemcpy(d_input, input, DATA_SIZE_X * DATA_SIZE_Y * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpyToSymbol(mask, mask_value, MASK_WIDTH * MASK_WIDTH * sizeof(int));

    dim3 dimBlock(TILE_WIDTH_X, TILE_WIDTH_Y);
    dim3 dimGrid((DATA_SIZE_X + dimBlock.x - 1) / dimBlock.x, (DATA_SIZE_Y + dimBlock.y - 1) / dimBlock.y);

    convolution_2d<<<dimGrid, dimBlock>>>(d_input, d_output, DATA_SIZE_X, DATA_SIZE_Y);

    cudaMemcpy(output, d_output, DATA_SIZE_X * DATA_SIZE_Y * sizeof(int), cudaMemcpyDeviceToHost);

    for (int j = 0; j < DATA_SIZE_Y; j++) {
        for (int i = 0; i < DATA_SIZE_X; i++)
            printf("%d ", output[j][i]);
        printf("\n");
    }

    cudaFree(d_input);
    cudaFree(d_output);

    return 0;
}
